# Feature Selection

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import featuretools as ft
import scipy.stats as ss
import math 

from scipy import stats
from collections import Counter
from pathlib import Path

%matplotlib inline

sns.set(style="ticks")

## Dataset

In [5]:
DATASET = Path("datasets/lotto/data_processed.csv")

# The Answer to the Ultimate Question of Life, the Universe, and Everything.
np.random.seed(42)